In [1]:
#ipython magic
%reset -f
%matplotlib notebook
%load_ext autoreload
%autoreload 2

ERROR:root:Invalid alias: The name clear can't be aliased because it is another magic command.
ERROR:root:Invalid alias: The name more can't be aliased because it is another magic command.
ERROR:root:Invalid alias: The name less can't be aliased because it is another magic command.
ERROR:root:Invalid alias: The name man can't be aliased because it is another magic command.


In [2]:
#general imports
import sys
sys.path.append('..')
import os
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import math
import time

In [3]:
sys.path.append('/home/jamesrowland/Documents/Code/Vape')

In [4]:
#notebook specific imports
from utils.artifact_removal import artifact_removal
from utils.utils_funcs import threshold_detect, stim_start_frame, paq_data
import tifffile
from utils.paq2py import paq_read
import copy


In [5]:
#global plotting params
params = {'legend.fontsize': 'x-large',
         'axes.labelsize': 'x-large',
         'axes.titlesize':'x-large',
         'xtick.labelsize':'x-large',
         'ytick.labelsize':'x-large'}
plt.rcParams.update(params)
sns.set()
sns.set_style('white')

In [6]:
# tiff_path = r'/home/jamesrowland/Documents/packerstation/rlees/Data/2019-04-25/RL036/2019-04-25_RL036_t-001/2019-04-25_RL036_t-001_Cycle00001_Ch3.tif'
# naparm_path = r'/home/jamesrowland/Documents/packerstation/rlees/Data/2019-04-25/RL036/2019-04-25_RL036_NAPARM'
# paq_path = r'/home/jamesrowland/Documents/packerstation/rlees/Data/2019-04-25/RL036/2019-04-25_RL036_t-001.paq'

In [7]:
tiff_path = '/home/jamesrowland/Documents/packerstation/jrowland/Data/2019-06-29/2019-06-29_J048_t-001/2019-06-29_J048_t-001_Cycle00001_Ch3.tif'
#naparm_path = 
paq_path = '/home/jamesrowland/Documents/packerstation/jrowland/Data/2019-06-29/2019-06-29_J048_t001.paq'

In [8]:
stack = tifffile.imread(tiff_path)
#stack = stack[0:1000,:,:]
stack_original = copy.deepcopy(stack)
stack.shape

(5228, 512, 512)

In [9]:
paq = paq_read(paq_path)
stim_frames = stim_start_frame(paq, 'markpoints2packio')
frame_clock = paq_data(paq, 'frame_clock', threshold_ttl=True)
stim_frames

array([ 511409,  532163,  552248,  573002,  593754,  613839,  634594,
        654676,  675431,  695516,  716268,  736353,  757107,  777192,
        797944,  818029,  838784,  859536,  879621,  900375,  920458,
        941213,  961298,  982052, 1002135, 1022889, 1042974, 1063726,
       1083811, 1104566, 1125318, 1145403, 1166157, 1186240, 1206994,
       1227080, 1247834, 1267917, 1288671, 1308756, 1329508, 1349593,
       1370348, 1391100, 1411185, 1431940, 1452022, 1472777, 1492862,
       1513614, 1533699, 1554453, 1574536, 1595291, 1615376, 1636128,
       1656882, 1676967, 1697722, 1717804, 1738559, 1758644, 1779396,
       1799481, 1820236, 1840318, 1861073, 1881158, 1901910, 1922665,
       1942750, 1963502, 1983587, 2004342, 2024424, 2045179, 2065264,
       2086016, 2106101, 2126856, 2147608, 2167693, 2188448, 2208530,
       2229285, 2249370, 2270122, 2290207, 2310962, 2331044, 2351799,
       2371884, 2392636, 2412721, 2433476, 2454228, 2474313, 2495068,
       2515150, 2535

In [10]:
duration_ms = 250 #length of stim
duration_samples = (duration_ms / 1000) * paq['rate']

In [11]:
#use this to remove frames during stim based on paq
to_remove = []

for stim_frame in stim_frames: 

    # frame indexes that fall during the stim
    in_stim = np.where((frame_clock >= stim_frame) & (frame_clock <= stim_frame + duration_samples))[0]
    
    #empircal observation, these are the frames with artifact
    in_stim = np.append(in_stim, in_stim[-1]+1)
    in_stim = np.append(in_stim, in_stim[-1]+1)
    
    to_remove.append(in_stim)

to_remove = np.ravel(to_remove)

In [12]:
# this just chops the frames out without artifact removal

stack_removed = np.delete(stack, to_remove, axis=0)
tifffile.imwrite('test_removed.tiff', stack_removed)
tifffile.imwrite('test.tiff', stack_original)

In [13]:
stack.shape

(5228, 512, 512)

In [14]:
1/0

ZeroDivisionError: division by zero

In [ ]:
t1 = time.time()

to_remove = [i for i in to_remove if i < stack.shape[0]]

stack_removed = artifact_removal(stack, remove_me=to_remove, width_thresh=5)

t2 = time.time()

t2-t1

In [ ]:

tifffile.imwrite('test_removed.tiff', stack_removed)
tifffile.imwrite('test.tiff', stack_original)

In [ ]:
1/0

In [ ]:
len(to_remove)

In [ ]:
to_remove 

In [ ]:
from artifact_removal import artifact_removal
import tifffile

stack = tifffile.imread(path_to_stack)
remove_me = list_of_frames_with_artifact_in

cleaned_stack = artifact_removal(stack, remove_me=remove_me, width_thresh=5)


In [ ]:
(60/500) * 1000